In [2]:
import torch
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("data/nhs_corona.csv")

In [4]:
df.head()

,Unnamed: 0,date,url,title,format,text
0,0,2020-03-20,https://www.england.nhs.uk/coronavirus/,NaN,text/html; charset=UTF-8,Coronavirus\nSkip to main content\nCookies on ...
1,1,2020-03-20,https://www.england.nhs.uk/accessibility/,NaN,text/html; charset=UTF-8,NHS England » About accessibility\nSkip to mai...
2,2,2020-03-20,https://www.england.nhs.uk/coronavirus/publica...,NaN,text/html; charset=UTF-8,Coronavirus » Letter: Covid-19 and professiona...
3,3,2020-03-20,https://www.gov.uk/government/publications/cov...,NaN,text/html; charset=UTF-8,COVID-19: guidance for households with possibl...
4,4,2020-03-20,https://www.gov.uk/government/organisations,NaN,text/html; charset=UTF-8,"Departments, agencies and public bodies - GOV...."


### Helper for text

In [40]:
from bs4 import BeautifulSoup 

import spacy
nlp = spacy.load("en_core_web_sm")

In [49]:
def get_sentences(html):
    soup = BeautifulSoup(html) 
    spacy_text =  nlp(soup.get_text())
    return spacy_text.sents

sentences = list(get_sentences(df['text'].values[0]))

In [50]:
sentences

[Coronavirus
 Skip to main content
 Cookies on the NHS England and NHS Improvement website,
 We’ve put some small files called cookies on your device to make our site work.,
 We’d also like to use analytics cookies.,
 These send information about how our site is used to a service called Google Analytics.,
 We use this information to improve our site.,
 Let us know if this is OK.,
 We’ll use a cookie to save your choice.,
 You can read more about our cookies before you choose.,
 Change my preferences,
 I'm OK with analytics cookies
 NHS England and NHS Improvement coronavirus
 Search
 Search
 Menu,
 Our advice for clinicians on the coronavirus is here.,
 If you are a member of the public looking for health advice, go to the NHS website.,
 And if you are looking for the latest travel information, and advice about the government response to the outbreak, go to the gov.uk website.,
 Last updated on:,
 20 March, 2020 10:55 am
 Coronavirus guidance for clinicians
 Primary care
 Secondary car

### BERT

In [26]:
from transformers import *

pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
BERT = BertModel.from_pretrained(pretrained_weights)

In [37]:
def embed_text(text):
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    with torch.no_grad():
        last_hidden_states = BERT(input_ids)[0]  
        return last_hidden_states.detach().numpy()

In [39]:
embed_text("hello world").shape

(1, 4, 768)

In [36]:
embeds = []

"Coronavirus\nSkip to main content\nCookies on the NHS England and NHS Improvement website\nWe’ve put some small files called cookies on your device to make our site work.\nWe’d also like to use analytics cookies. These send information about how our site is used to a service called Google Analytics. We use this information to improve our site.\nLet us know if this is OK. We’ll use a cookie to save your choice. You can\xa0read more about our cookies before you choose.\nChange my preferences\nI'm OK with analytics cookies\nNHS England and NHS Improvement coronavirus\nSearch\nSearch\nMenu\nOur advice for clinicians on the coronavirus is here. If you are a member of the public looking for health advice, go to the NHS website. And if you are looking for the latest travel information, and advice about the government response to the outbreak, go to the gov.uk website.\nLast updated on: 20 March, 2020 10:55 am\nCoronavirus guidance for clinicians\nPrimary care\nSecondary care\nCommunity healt